In [1]:
import spacy.cli
spacy.cli.download("en_core_web_md")

import pandas as pd
import string
import spacy
import nltk
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from nltk.corpus import stopwords
import en_core_web_md
nltk.download('wordnet')
nltk.download('stopwords')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 32.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/Users/pgalli/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
[nltk_data] Downloading package wordnet to /Users/pgalli/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pgalli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
data = pd.read_csv('reddit_data_full.csv')

In [3]:
nlp = en_core_web_md.load(disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):
    '''This function converts terms to their base forms.'''
    output = []
    for sent in texts:
        doc = nlp(str(sent))
        output.append(
            [token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return output


text_list = data['clean_post'].tolist()
tokenized_text = lemmatization(text_list)

In [4]:
dictionary = corpora.Dictionary(tokenized_text)
if len(dictionary) > 0:
    doc_term_matrix = [dictionary.doc2bow(text) for text in tokenized_text]
else:
    doc_term_matrix = []

## 10 Topics

In [5]:
if doc_term_matrix:
    LDA = gensim.models.ldamodel.LdaModel
    lda_model = LDA(
        corpus=doc_term_matrix,
        id2word=dictionary,
        num_topics=10,
        random_state=100,
        chunksize=1000,
        passes=50,
        iterations=100
    )
    print(lda_model.print_topics())
else:
    print("Document term matrix is empty, cannot build LDA model.")

[(0, '0.074*"life" + 0.052*"people" + 0.032*"fucking" + 0.021*"world" + 0.020*"shit" + 0.020*"fuck" + 0.014*"thing" + 0.012*"point" + 0.012*"way" + 0.011*"person"'), (1, '0.104*"job" + 0.098*"work" + 0.016*"hour" + 0.015*"money" + 0.014*"home" + 0.012*"new" + 0.011*"motivation" + 0.010*"energy" + 0.009*"day" + 0.007*"task"'), (2, '0.030*"year" + 0.026*"time" + 0.023*"friend" + 0.021*"life" + 0.015*"day" + 0.015*"thing" + 0.015*"family" + 0.014*"good" + 0.010*"month" + 0.010*"bad"'), (3, '0.092*"pain" + 0.040*"pill" + 0.026*"goodbye" + 0.016*"painless" + 0.016*"emptiness" + 0.014*"rope" + 0.012*"unbearable" + 0.011*"dose" + 0.011*"prescription" + 0.009*"body"'), (4, '0.047*"people" + 0.039*"thing" + 0.030*"friend" + 0.024*"time" + 0.019*"depression" + 0.017*"happy" + 0.017*"feeling" + 0.016*"depressed" + 0.014*"bad" + 0.014*"good"'), (5, '0.075*"money" + 0.040*"food" + 0.035*"car" + 0.028*"bill" + 0.025*"tooth" + 0.023*"homeless" + 0.021*"pay" + 0.013*"rent" + 0.012*"dollar" + 0.010*"di

In [6]:
total_docs = len(doc_term_matrix)
if total_docs > 0:
    print('\nPerplexity:', lda_model.log_perplexity(
        doc_term_matrix, total_docs=total_docs))
    coherence_model_lda = CoherenceModel(
        model=lda_model,
        texts=tokenized_text,
        dictionary=dictionary,
        coherence='c_v'
    )
    coherence_lda = coherence_model_lda.get_coherence()
    print('Coherence:', coherence_lda)
else:
    print("No documents to evaluate coherence or perplexity.")


Perplexity: -7.829607360960524
Coherence: 0.451284553832768


In [8]:
if total_docs > 0:
    pyLDAvis.enable_notebook()
    vis_data = gensimvis.prepare(lda_model, doc_term_matrix, dictionary)
    vis_data
    pyLDAvis.save_html(vis_data, '10_reddit_lda_visualization.html')
else:
    print("No documents for visualization.")

## 20 Topics

In [9]:
if doc_term_matrix:
    LDA = gensim.models.ldamodel.LdaModel
    lda_model = LDA(
        corpus=doc_term_matrix,
        id2word=dictionary,
        num_topics=20,
        random_state=100,
        chunksize=1000,
        passes=50,
        iterations=100
    )
    print(lda_model.print_topics())
else:
    print("Document term matrix is empty, cannot build LDA model.")

[(0, '0.182*"people" + 0.065*"world" + 0.043*"human" + 0.028*"person" + 0.024*"other" + 0.023*"society" + 0.019*"message" + 0.019*"selfish" + 0.016*"shitty" + 0.015*"existence"'), (1, '0.353*"suicide" + 0.048*"note" + 0.024*"plan" + 0.019*"hotline" + 0.017*"temporary" + 0.014*"mechanism" + 0.013*"question" + 0.012*"track" + 0.012*"police" + 0.012*"inevitable"'), (2, '0.037*"friend" + 0.031*"time" + 0.030*"thing" + 0.027*"people" + 0.019*"good" + 0.019*"bad" + 0.014*"year" + 0.013*"life" + 0.012*"way" + 0.012*"person"'), (3, '0.082*"depression" + 0.067*"mental" + 0.044*"help" + 0.043*"health" + 0.038*"anxiety" + 0.034*"therapy" + 0.027*"doctor" + 0.026*"medication" + 0.026*"therapist" + 0.025*"bad"'), (4, '0.036*"ugly" + 0.035*"loneliness" + 0.031*"weight" + 0.029*"amp" + 0.025*"hair" + 0.023*"personality" + 0.022*"body" + 0.021*"medium" + 0.020*"fat" + 0.020*"attractive"'), (5, '0.109*"tomorrow" + 0.109*"ready" + 0.063*"episode" + 0.049*"depressive" + 0.047*"tooth" + 0.033*"shower" + 0

In [10]:
total_docs = len(doc_term_matrix)
if total_docs > 0:
    print('\nPerplexity:', lda_model.log_perplexity(
        doc_term_matrix, total_docs=total_docs))
    coherence_model_lda = CoherenceModel(
        model=lda_model,
        texts=tokenized_text,
        dictionary=dictionary,
        coherence='c_v'
    )
    coherence_lda = coherence_model_lda.get_coherence()
    print('Coherence:', coherence_lda)
else:
    print("No documents to evaluate coherence or perplexity.")


Perplexity: -9.143902610364657
Coherence: 0.40729784207988373


In [12]:
if total_docs > 0:
    pyLDAvis.enable_notebook()
    vis_data = gensimvis.prepare(lda_model, doc_term_matrix, dictionary)
    vis_data
    pyLDAvis.save_html(vis_data, '20_reddit_lda_visualization.html')
else:
    print("No documents for visualization.")

## 30 Topics

In [13]:
if doc_term_matrix:
    LDA = gensim.models.ldamodel.LdaModel
    lda_model = LDA(
        corpus=doc_term_matrix,
        id2word=dictionary,
        num_topics=30,
        random_state=100,
        chunksize=1000,
        passes=50,
        iterations=100
    )
    print(lda_model.print_topics())
else:
    print("Document term matrix is empty, cannot build LDA model.")

[(17, '0.097*"dog" + 0.047*"insurance" + 0.045*"nightmare" + 0.044*"bill" + 0.041*"medical" + 0.037*"pay" + 0.032*"debt" + 0.030*"symptom" + 0.027*"ok" + 0.027*"function"'), (2, '0.044*"guy" + 0.042*"girl" + 0.032*"woman" + 0.030*"man" + 0.020*"sex" + 0.016*"date" + 0.016*"relationship" + 0.014*"love" + 0.014*"drunk" + 0.014*"old"'), (6, '0.042*"exhausted" + 0.041*"deep" + 0.036*"hole" + 0.033*"bipolar" + 0.028*"bottle" + 0.025*"light" + 0.025*"black" + 0.023*"relief" + 0.020*"dark" + 0.019*"bpd"'), (12, '0.166*"normal" + 0.097*"attack" + 0.089*"panic" + 0.038*"rock" + 0.035*"urge" + 0.033*"weird" + 0.033*"bottom" + 0.028*"overwhelmed" + 0.027*"song" + 0.022*"torture"'), (16, '0.190*"kid" + 0.075*"child" + 0.070*"wife" + 0.023*"emptiness" + 0.022*"grandma" + 0.021*"service" + 0.021*"married" + 0.019*"fire" + 0.017*"rejection" + 0.015*"spot"'), (24, '0.132*"pain" + 0.049*"way" + 0.049*"death" + 0.045*"suicide" + 0.044*"alive" + 0.040*"dead" + 0.037*"end" + 0.036*"life" + 0.033*"kill" + 

In [14]:
total_docs = len(doc_term_matrix)
if total_docs > 0:
    print('\nPerplexity:', lda_model.log_perplexity(
        doc_term_matrix, total_docs=total_docs))
    coherence_model_lda = CoherenceModel(
        model=lda_model,
        texts=tokenized_text,
        dictionary=dictionary,
        coherence='c_v'
    )
    coherence_lda = coherence_model_lda.get_coherence()
    print('Coherence:', coherence_lda)
else:
    print("No documents to evaluate coherence or perplexity.")


Perplexity: -10.119497604980612
Coherence: 0.39603295433127383


In [15]:
if total_docs > 0:
    pyLDAvis.enable_notebook()
    vis_data = gensimvis.prepare(lda_model, doc_term_matrix, dictionary)
    vis_data
    pyLDAvis.save_html(vis_data, '30_reddit_lda_visualization.html')
else:
    print("No documents for visualization.")